### Minimal code to perform text-to-audio (tta)

In [1]:
import os
import torch
import numpy as np
from audioldm2 import text_to_audio, build_model, save_wave, get_time

# # Set up environment
# os.environ["TOKENIZERS_PARALLELISM"] = "true"
# torch.set_float32_matmul_precision("high")

C:\Users\dslee\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\dslee\anaconda3\envs\audioldm\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# Create save path
model_name = "audioldm_48k"  # Choose the speech model
device = "cuda" if torch.cuda.is_available() else "cpu"
save_path = "./output"  # Set save path
# save_path = os.path.join(save_path, get_time())
os.makedirs(save_path, exist_ok=True)

# Load the model
audioldm2 = build_model(model_name=model_name, device=device)

Loading AudioLDM-2: audioldm_48k
Loading model on cuda


c:\Users\dslee\anaconda3\envs\audioldm\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\dslee\anaconda3\envs\audioldm\lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
c:\Users\dslee\anaconda3\envs\audioldm\lib\site-packages\torchaudio\transforms\_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module

+ Use extra condition on UNet channel using Film. Extra condition dimension is 512. 
DiffusionWrapper has 262.70 M params.


c:\Users\dslee\anaconda3\envs\audioldm\lib\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
d:\projects\AudioLDM2\audioldm2\pipeline.py:172: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights

In [3]:
# Define parameters for TTS
text = "The sound of baby crying."
transcription = ""
duration = 10  # Duration in seconds; because all the training samples were 10s, the generated samples should be 10s long, otherwise it results in poor audio quality.
guidance_scale = 3.5  # Guidance scale for text adherence
ddim_steps = 200  # DDIM sampling steps
n_candidate_gen_per_text = 3  # Number of candidate generations per text
random_seed = np.random.randint(999999)
sample_rate = 48000 if '48k' in model_name else 16000
latent_t_per_second = 12.8 if '48k' in model_name else 25.6

In [4]:
# Generate TTS audio
waveform = text_to_audio(
    audioldm2,
    text=text,
    transcription=transcription,  # To guide the model based on the transcription
    seed=random_seed,
    duration=duration,
    guidance_scale=guidance_scale,
    ddim_steps=ddim_steps,
    n_candidate_gen_per_text=n_candidate_gen_per_text,
    batchsize=1,  # Single batch for simplicity
    latent_t_per_second=latent_t_per_second
)

# Save the generated audio
save_wave(waveform, save_path, name=f"TTS_{text[:10]}", samplerate=sample_rate)

# Inform the user about the save location
print(f"Audio generated and saved at: {save_path}")


Running DDIM Sampling with 200 timesteps


DDIM Sampler: 100%|██████████| 200/200 [00:55<00:00,  3.58it/s]


INFO: clap model calculate the audio embedding as condition
Similarity between generated audio and text:
0.49 0.42 0.47
Choose the following indexes as the output: [0]
Save audio to ./output\TTS_The sound .wav
Audio generated and saved at: ./output
